# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [10]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/zinth/production/05_src/data/adult/adult.data', header=None, names=columns)
              .assign(income=lambda x: (x.income.str.strip() == '>50K') * 1))

print (adult_dt)


       age          workclass  fnlwgt    education  education-num  \
0       39          State-gov   77516    Bachelors             13   
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

            marital-status          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   White   
1       Married-civ-spo

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Separate features (X) and target (Y)
X = adult_dt.drop(columns=['income'])  # All columns except 'income'
Y = adult_dt['income']                 # Only the 'income' column

# Split into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print(train_test_split)


<function train_test_split at 0x000001D8D5FFC550>


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

1.The random state in the train_test_split function is a seed value that initializes the random number generator. This ensures that the data split or any other random operation produces the same output each time the code is run, making the process reproducible.

2. Reproducibility: By setting a specific random_state, we ensure that the data split is consistent across different runs. 
Fair Comparisons:Fixing the random state allows for a fair comparison across models or experiments. 
Debugging:When something goes wrong during model training or evaluation, knowing the exact data split can make debugging easier. 


# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Define your numerical and categorical columns
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Create the preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Define the pipeline
model_pipeline = Pipeline([
    ('preprocessing', preprocessor),           # Preprocessing step using the ColumnTransformer
    ('classifier', RandomForestClassifier())   # Classification step using RandomForestClassifier
])


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [36]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Cross-validation
cv_results = cross_validate(
    model_pipeline,
    X_train,
    Y_train,
    cv=5,
    scoring=['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy'],
    return_train_score=True
)

c:\Users\zinth\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\zinth\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [37]:
# Convert the results to a DataFrame for better visualization
results_df = pd.DataFrame(cv_results)

# Sort by negative log loss of the test set
results_df = results_df.sort_values(by='test_neg_log_loss')


Calculate the mean of each metric. 

In [38]:
# Calculate mean of each metric
mean_results = results_df.mean()

# Display fold-level results
print("Fold-level results:")
print(results_df)

# Display mean results
print("\nMean results across folds:")
print(mean_results)

Fold-level results:
    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4  13.015368    0.234282          -0.387412           -0.081284      0.902338   
1  12.552821    0.233564          -0.387200           -0.080961      0.902274   
3  12.265246    0.218725          -0.362471           -0.082193      0.907618   
0  12.360578    0.233698          -0.361766           -0.081397      0.905452   
2  13.297366    0.218699          -0.352148           -0.081430      0.903561   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4            1.0       0.857174        0.999945                0.779275   
1            1.0       0.847774        1.000000                0.767378   
3            1.0       0.858052        1.000000                0.781392   
0            1.0       0.848870        1.000000                0.770253   
2            1.0       0.853664        1.000000                0.774496   

   train_balanced_accuracy  
4            

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [39]:
# Evaluate performance on the testing data
model_pipeline.fit(X_train, Y_train)  # Fit the model to the training data
Y_pred_proba = model_pipeline.predict_proba(X_test)  # Get prediction probabilities

# Calculate performance metrics on the test set
neg_log_loss_test = log_loss(Y_test, Y_pred_proba)
roc_auc_test = roc_auc_score(Y_test, Y_pred_proba[:, 1])
accuracy_test = accuracy_score(Y_test, model_pipeline.predict(X_test))
balanced_accuracy_test = balanced_accuracy_score(Y_test, model_pipeline.predict(X_test))

# Store results in a dictionary
test_results = {
    'negative_log_loss': neg_log_loss_test,
    'roc_auc': roc_auc_test,
    'accuracy': accuracy_test,
    'balanced_accuracy': balanced_accuracy_test
}

# Display test results
print("\nTest results:")
print(test_results)


Test results:
{'negative_log_loss': 0.36389912182016165, 'roc_auc': 0.9023113037197544, 'accuracy': 0.8551540587572934, 'balanced_accuracy': 0.7744181307957222}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable, in this case, the income column, immediately after loading the data is convenient for several reasons:

Simplification for Modeling: By converting the income variable into a binary format (0 and 1), we make it easier to use in machine learning models, especially classification algorithms. 
Clear Interpretation: Recoding the income variable helps clarify the relationship we are modeling. With 0 representing income less than or equal to 50K and 1 representing income greater than 50K.
Data Consistency: Stripping whitespace with x.income.str.strip() ensures that any leading or trailing spaces in the data do not affect the comparison.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.